In [1]:
#ignition delay for RMG model of propane 
import pandas as pd
import numpy as np
import time
import cantera as ct
import sys 

In [2]:
print(f"Runnning Cantera version: {ct.__version__}")

#constants
pressure = 101325 #Pascals
mole_frac_dict = {"C3H8(1)": 1, "O2(2)": 5, "Ar":94} #this technically sets equivalence ratio to 1 
#mole_frac_dict_eduardos = {"C3H8(1)": 1, "O2(2)": 6, "Ar":93}

def ignition_delay(states, species): 
    """
    This function computes the ignition delay from the occurence of the peak in species' concentration
    """
    # pd = states.to_pandas()
    # print(pd)
    i_ign = states(species).Y.argmax()
    return states.t[i_ign]

def gas_conditions(temp): 
    """
    define gas conditions
    """
    gas.TPX = temp, pressure, mole_frac_dict

Runnning Cantera version: 2.5.1


In [8]:
#to save data
taus = []


reference_species = "OH(6)"

x_axis_vals= list(np.linspace(0.55, 0.8, 20))


for i in x_axis_vals: 
        
    gas = ct.Solution("./cantera/chem_annotated.yaml")
    temp= 1000/i
    gas_conditions(temp)
    
    # Create a batch reactor object and add it to a reactor network
    # In this example, the batch reactor will be the only reactor in the network
    r = ct.IdealGasReactor(contents=gas, name="Batch Reactor")
    reactor_network = ct.ReactorNet([r])
    
    
    # use the above list to create a DataFrame
    time_history = ct.SolutionArray(gas, extra="t")

    t0 = time.time()
    
    # This is a starting estimate. If you do not get an ignition within this time, increase it
    estimated_ignition_delay_time = 0.4
    t = 0

    counter = 1
    while t < estimated_ignition_delay_time:
        t = reactor_network.step()
        if not counter % 10:
            # We will save only every 10th value. Otherwise, this takes too long
            # Note that the species concentrations are mass fractions
            time_history.append(r.thermo.state, t=t)
        counter += 1

    # We will use the 'OH(6)' species to compute the ignition delay
    tau = ignition_delay(time_history, reference_species)

    # Toc
    t1 = time.time()

    print(f"Computed Ignition Delay: {tau:.3e} seconds. Took {t1-t0:3.2f}s to compute")

    #save to large data array
    taus.append(tau)
    
    # If you want to save all the data - molefractions, temperature, pressure, etc
    #time_history.to_pandas().to_csv("time_history.csv")


Computed Ignition Delay: 1.594e-04 seconds. Took 2.42s to compute
Computed Ignition Delay: 1.628e-04 seconds. Took 2.02s to compute
Computed Ignition Delay: 1.697e-04 seconds. Took 2.04s to compute
Computed Ignition Delay: 1.677e-04 seconds. Took 1.92s to compute
Computed Ignition Delay: 1.774e-04 seconds. Took 2.15s to compute
Computed Ignition Delay: 1.907e-04 seconds. Took 2.02s to compute
Computed Ignition Delay: 2.126e-04 seconds. Took 2.23s to compute
Computed Ignition Delay: 2.456e-04 seconds. Took 2.36s to compute
Computed Ignition Delay: 2.916e-04 seconds. Took 2.50s to compute
Computed Ignition Delay: 3.554e-04 seconds. Took 2.38s to compute
Computed Ignition Delay: 4.462e-04 seconds. Took 2.42s to compute
Computed Ignition Delay: 5.912e-04 seconds. Took 2.19s to compute
Computed Ignition Delay: 8.008e-04 seconds. Took 2.42s to compute
Computed Ignition Delay: 1.102e-03 seconds. Took 2.52s to compute
Computed Ignition Delay: 1.548e-03 seconds. Took 2.54s to compute
Computed I

In [48]:
#save to csv file 
dataframe = pd.DataFrame([x_axis_vals, taus]) 
dataframe.to_csv('C3H8_ignition_delays_eds_stoic.csv', index=False)

In [9]:
#save to csv file 
dataframe = pd.DataFrame([x_axis_vals, taus]) 
dataframe.to_csv('C3H8_ignition_delays.csv', index=False)

In [11]:
nora_df = pd.read_csv('/work/westgroup/nora/Code/projects/PFAS/models/C3H8_CH2F2/C3H8/C3H8_sevys_settings_branching/C3H8_ignition_delays.csv')
nora_df.iloc[1].values

array([0.00015943, 0.0001628 , 0.00016973, 0.00016766, 0.00017743,
       0.00019072, 0.00021262, 0.00024564, 0.00029165, 0.00035538,
       0.00044616, 0.00059118, 0.00080084, 0.0011016 , 0.00154764,
       0.00220938, 0.003174  , 0.00457551, 0.00659793, 0.0094997 ])